# Driver Drowsiness Detection System

In [1]:
# import the necessary libraries
import dlib
import cv2
import multiprocessing
from math import hypot
from playsound import playsound


class Project:
    def __init__(self):     # Constructor
        self.p = multiprocessing.Process()

    def soundOn(self):      # Play Sound
        if not self.p.is_alive():
            self.p = multiprocessing.Process(target=playsound, args=("alarm.mp3",))
            self.p.start()

    def soundOff(self):     # Stop Sound
        if self.p.is_alive():
            self.p.terminate()

    def aspect_ratio(self, landmark_list, face_landmark):   # Calculate Aspect Ratio
        n = len(landmark_list)
        left_point = (face_landmark.part(landmark_list[0]).x, face_landmark.part(landmark_list[0]).y)
        right_point = (face_landmark.part(landmark_list[n//2]).x, face_landmark.part(landmark_list[n//2]).y)
        # Calculate Horizontal Length
        hor_length = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))

        top = list()    # Co-ordinates of the upper part    
        bottom = list() # Co-ordinates of the lower part 

        for i in range(1, n//2):
            top.append((face_landmark.part(landmark_list[i]).x, face_landmark.part(landmark_list[i]).y))
            bottom.append((face_landmark.part(landmark_list[-1*i]).x, face_landmark.part(landmark_list[-1*i]).y))

        ver_lengths = list()
        for i in range(len(top)):   # Calculate vertical lengths
            ver_lengths.append(hypot((top[i][0] - bottom[i][0]), (top[i][1] - bottom[i][1])))

        s = len(ver_lengths)
        ratio = sum(ver_lengths) / (s * hor_length) # Calculate aspect ratio
        return ratio

    def drowsinessDetectionSystem(self, video_path = 0):
        cap = cv2.VideoCapture(video_path)
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

        eye_close_count = 0
        mouth_open_count = 0
        font = cv2.FONT_HERSHEY_COMPLEX

        yawn_count = 0
        yawning = False
        drowsy_count = 0 
        drowsy = False

        max_ear = 0
        min_ear = 1
        per = 35
        threshold_ear = 0.25
        threshold_mar = 0.35
        max_frame_count = 20

        while True:
            ret, frame = cap.read()
            if not ret:     # If no frame is detected then stop
                break

            frame = cv2.flip(frame, 1)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Convert the image to gray scale
            faces = detector(gray)  # Detect faces and get the co-ordinates of the rectangle where face is detected

            if 0 < len(faces):
                face = faces[0]     # Consider only 1st detected face
                face_landmark = predictor(gray, face)   # Get facial landmarks

                left_eye_ratio = self.aspect_ratio(list(range(36, 42)), face_landmark)
                right_eye_ratio = self.aspect_ratio(list(range(42, 48)), face_landmark)
                eye_aspect_ratio = (left_eye_ratio + right_eye_ratio) / 2   # Calculate eye aspect ratio
                cv2.putText(frame, "Cur EAR: "+str(round(eye_aspect_ratio, 5)), (10, 15), font, 0.5, (0, 0, 0))
                cv2.putText(frame, "Thresh EAR: "+str(round(threshold_ear, 5)), (10, 30), font, 0.5, (0, 0, 0))

                cv2.putText(frame, "Drowsy Count: "+str(drowsy_count), (250, 15), font, 0.5, (0, 0, 0))

                inner_lip_ratio = self.aspect_ratio(list(range(60, 68)), face_landmark)
                outter_lip_ratio = self.aspect_ratio(list(range(48, 60)), face_landmark)
                mouth_aspect_ratio = (inner_lip_ratio + outter_lip_ratio) / 2 # Calculate mouth aspect ratio
                cv2.putText(frame, "Cur MAR: "+str(round(mouth_aspect_ratio, 5)), (475, 15), font, 0.5, (0, 0, 0))
                cv2.putText(frame, "Thresh MAR: "+str(round(threshold_mar, 5)), (475, 30), font, 0.5, (0, 0, 0))
                cv2.putText(frame, "Yawn Count: "+str(yawn_count), (475, 45), font, 0.5, (0, 0, 0))

                max_ear = max(max_ear, eye_aspect_ratio)
                min_ear = min(min_ear, eye_aspect_ratio)
                diff = max_ear - min_ear
                threshold_ear = min_ear + diff * per / 100      # Calculate threshold eye aspect ratio

                if eye_aspect_ratio < threshold_ear:
                    eye_close_count += 1
                else:
                    eye_close_count = 0

                if mouth_aspect_ratio > threshold_mar:
                    mouth_open_count += 1
                else:
                    mouth_open_count = 0

                # Get co-ordinates of rectangle in which face is detected
                x1, y1 = face.left(), face.top()
                x2, y2 = face.right(), face.bottom()

                if eye_close_count >= max_frame_count:
                    cv2.putText(frame, "Eyes Closed", (10, 55), font, 0.75, (0, 0, 255))
                if mouth_open_count >= max_frame_count:
                    cv2.putText(frame, "Yawning", (475, 70), font, 0.75, (0, 0, 255))
                    if not yawning:
                        yawn_count += 1
                        yawning = True
                else:
                    if yawning:
                        yawning = False

                if eye_close_count < max_frame_count and mouth_open_count < max_frame_count:
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (31, 163, 21), 2)
                    cv2.putText(frame, "Active :)", (250, 50), font, 1, (31, 163, 21))
                    self.soundOff()
                    if drowsy:
                        drowsy = False
                else:
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(frame, "Drowsy!", (250, 50), font, 1, (255, 0, 0))
                    self.soundOn()
                    if not drowsy:
                        drowsy_count += 1
                        drowsy = True

                for i in range(0, 68):  # Plot 68 facial landmarks
                    (x, y) = (face_landmark.part(i).x, face_landmark.part(i).y)
                    cv2.circle(frame, (x, y), 1, (255, 255, 255), -1)

            else:
                self.soundOff()

            cv2.imshow("Driver Drowsiness Detection System", frame) # Display frame

            key = cv2.waitKey(1)    # Press Esc to exit
            if key == 27:
                break

        self.soundOff()
        cap.release()
        cv2.destroyAllWindows()

obj = Project()
obj.drowsinessDetectionSystem()